In [ ]:
import matplotlib.pyplot as plt
!apt-get install -y fonts-nanum > /dev/null
!fc-cache -fv > /dev/null
!rm -rf ~/.cache/matplotlib > /dev/null

In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rcParams['axes.unicode_minus'] = False
plt.rc('font', family='NanumBarunGothic')

In [ ]:
from google.colab import drive
drive.mount('/content/mnt')

Drive already mounted at /content/mnt; to attempt to forcibly remount, call drive.mount("/content/mnt", force_remount=True).


In [ ]:
data_dir = '/content/mnt/MyDrive/data/cctv'


In [ ]:
cctv = pd.read_csv(f'{data_dir}/서울시CCTV설치운영현황(자치구)_년도별_210731기준.csv', encoding='euc-kr',skiprows=1)

In [ ]:
pop = pd.read_csv(f'{data_dir}/서울시_주민등록인구_구별(통계).tsv',sep='\t')

In [ ]:
pop.columns = ['기간', '구분', '세대', '인구합계', '남자인구합계', '여자인구합계', '한국인인구합계',
               '한국인남자인구합계', '한국인여자인구합계', '등록외국인인구합계', '등록외국인남자인구합계',
               '등록외국인여자합계', '세대당인구', '65세이상고령인구']

In [ ]:
pop.drop(index=[0,1,2], inplace=True)
pop.reset_index(inplace=True)
pop.drop(columns='index', inplace=True)

In [ ]:
cctv.drop(columns='2021년', inplace=True)
cctv.구분[2] = '중구'
cctv.drop(index=0, inplace=True)
cctv.reset_index(inplace=True)
cctv.drop(columns='index', inplace=True)


In [ ]:
cctv.head()

,구분,총계,2012년 이전,2012년,2013년,2014년,2015년,2016년,2017년,2018년,2019년,2020년
0,종로구,"1,772",813,0,0,210,150,1,261,85,9,200
1,중구,"2,333",16,114,87,77,236,240,372,386,155,361
2,용산구,"2,383",34,71,234,125,221,298,351,125,307,617
3,성동구,"3,602",448,125,212,105,339,310,874,390,262,461
4,광진구,"2,588",35,57,100,187,98,52,675,465,712,175


In [ ]:
pop.head()

,기간,구분,세대,인구합계,남자인구합계,여자인구합계,한국인인구합계,한국인남자인구합계,한국인여자인구합계,등록외국인인구합계,등록외국인남자인구합계,등록외국인여자합계,세대당인구,65세이상고령인구
0,2021.2/4,종로구,"73,525","155,106","75,009","80,097","146,029","70,985","75,044","9,077","4,024","5,053",1.99,"27,605"
1,2021.2/4,중구,"62,897","132,259","64,526","67,733","123,016","60,019","62,997","9,243","4,507","4,736",1.96,"23,980"
2,2021.2/4,용산구,"111,650","240,665","116,858","123,807","226,378","108,948","117,430","14,287","7,910","6,377",2.03,"38,884"
3,2021.2/4,성동구,"134,542","295,767","143,927","151,840","289,162","141,042","148,120","6,605","2,885","3,720",2.15,"45,591"
4,2021.2/4,광진구,"166,957","355,306","170,624","184,682","342,481","165,233","177,248","12,825","5,391","7,434",2.05,"50,815"


In [ ]:
len(pop)

25

In [ ]:
for i in range(len(pop)):
  pop.loc[i][2:] = pop.loc[i][2:].str.replace(',','')
cctv['총계'] = cctv.총계.str.replace(',','')

In [ ]:
pop.head(2)

,기간,구분,세대,인구합계,남자인구합계,여자인구합계,한국인인구합계,한국인남자인구합계,한국인여자인구합계,등록외국인인구합계,등록외국인남자인구합계,등록외국인여자합계,세대당인구,65세이상고령인구
0,2021.2/4,종로구,73525,155106,75009,80097,146029,70985,75044,9077,4024,5053,1.99,27605
1,2021.2/4,중구,62897,132259,64526,67733,123016,60019,62997,9243,4507,4736,1.96,23980


In [ ]:
cctv.head(2)

,구분,총계,2012년 이전,2012년,2013년,2014년,2015년,2016년,2017년,2018년,2019년,2020년
0,종로구,1772,813,0,0,210,150,1,261,85,9,200
1,중구,2333,16,114,87,77,236,240,372,386,155,361


# 구별 외국인 인구수대비 cctv 설치수

In [ ]:
f_pop = pop[['구분', '등록외국인인구합계']]
f_pop.head()

,구분,등록외국인인구합계
0,종로구,9077
1,중구,9243
2,용산구,14287
3,성동구,6605
4,광진구,12825


In [ ]:
gu_cctvf = pd.merge(f_pop, cctv[['구분', '총계']] ,on='구분')

In [ ]:
gu_cctvf['외국인대비cctv비율'] = gu_cctvf.등록외국인인구합계.astype(int) / gu_cctvf.총계.astype(int)
gu_cctvf.head()

,구분,등록외국인인구합계,총계,외국인대비cctv비율
0,종로구,9077,1772,5.122460
1,중구,9243,2333,3.961852
2,용산구,14287,2383,5.995384
3,성동구,6605,3602,1.833703
4,광진구,12825,2588,4.955564


In [ ]:
import json
import folium

In [ ]:
gu_cctvf.index

Int64Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
            17, 18, 19, 20, 21, 22, 23, 24],
           dtype='int64')

In [ ]:
gu_cctvf.set_index('구분', inplace=True)

In [ ]:
geo_data = json.load(open(f'{data_dir}/seoul-gu-simple.json'))

center = [37.541, 126.986]

map = folium.Map(location=center, zoom_start=10)

folium.Choropleth(
    geo_data=geo_data,
    data=gu_cctvf.외국인대비cctv비율,
    columns=(gu_cctvf.index, gu_cctvf.외국인대비cctv비율),
    key_on='feature.id',
    fill_color='Reds',
    legend_name='외국인대비cctv비율'
).add_to(map)
title = '<h3 align="center" style="font-size:20px">서울 구별 외국인대비cctv비율</h3>'
map.get_root().html.add_child(folium.Element(title))     
map